In [29]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import log_loss, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier



In [31]:
X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv')

In [33]:
y = y['poverty_score_y'].to_numpy()

In [35]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [37]:

# Initialize Logistic Regression Model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.204311152764761

# kNN

In [57]:
knn = KNeighborsClassifier(n_neighbors=30)

# Perform 5-fold cross-validation
cv_scores = cross_val_score(knn, X, y, cv=5, scoring='accuracy')

# Print the accuracy for each fold
print("Cross-validation scores:", cv_scores)
print(f"Average cross-validation accuracy: {cv_scores.mean():.2f}")



Cross-validation scores: [0.16307404 0.13683224 0.10965323 0.13870665 0.1782364 ]
Average cross-validation accuracy: 0.15


In [59]:
knn.fit(X_train, y_train)
knn.predict_proba(X_test)

array([[0.03333333, 0.06666667, 0.16666667, ..., 0.06666667, 0.        ,
        0.        ],
       [0.        , 0.06666667, 0.2       , ..., 0.03333333, 0.        ,
        0.        ],
       [0.1       , 0.06666667, 0.16666667, ..., 0.06666667, 0.        ,
        0.        ],
       ...,
       [0.        , 0.06666667, 0.13333333, ..., 0.06666667, 0.        ,
        0.        ],
       [0.        , 0.        , 0.16666667, ..., 0.03333333, 0.        ,
        0.        ],
       [0.1       , 0.16666667, 0.16666667, ..., 0.03333333, 0.        ,
        0.        ]])

# Random Forest

In [63]:


# Set up a grid of hyperparameters to search over
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Initialize the Random Forest model
rf = RandomForestClassifier(random_state=42)

# Set up GridSearchCV to search over the hyperparameter grid
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')

# Fit GridSearchCV to the training data
grid_search.fit(X, y)

# Get the best parameters and the best score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_:.2f}")


Best parameters: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 100}
Best cross-validation accuracy: 0.19


In [67]:
# Initialize the Random Forest model
rf = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=5, random_state=42)

# Train the Random Forest model on the training data
rf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf.predict(X_test)

# Evaluate the model using accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy on the test set: {accuracy:.2f}")


Accuracy on the test set: 0.22


In [ ]:
# X_test

# Neural network

In [ ]:
from sklearn.neural_network import MLPClassifier

# Initialize the MLPClassifier for a neural network
# 'hidden_layer_sizes' defines the number of neurons in the hidden layers (e.g., 100 neurons in one hidden layer)
# 'max_iter' is the maximum number of iterations for training
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42, solver='adam')

# Train the model
mlp.fit(X_train, y_train)

# Make predictions on the test set
y_pred = mlp.predict(X_test)

# Evaluate the model using accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy on the test set: {accuracy:.2f}")


# Do it for reals!

In [69]:
X_test = pd.read_csv('X_TEST.csv')

In [71]:
X.head()

,literate,attended_school,highest_school_lvl,preschool,finish_school_age,school_transportation_time,school_transportation_cost,school_expenses,is_student,not_attend_reason1_2.0,...,fam_role_8,fam_role_9,fam_role_10,sex,age,time_away,lives_with_partner,lives_with_parent,parents_education,num_parents_alive
0,True,1,2.0,0.0,16.04,40.0,0.0,140000.0,True,False,...,False,False,False,True,19,0,False,True,2.19,0
1,True,1,2.0,0.0,16.04,15.0,0.0,75000.0,True,False,...,False,False,False,True,19,0,False,True,2.19,0
2,True,1,2.0,0.0,16.04,6.0,0.0,100000.0,True,False,...,False,False,False,False,19,0,False,False,3.00,2
3,True,1,5.0,2.0,16.04,20.0,0.0,730000.0,True,False,...,False,False,False,True,20,0,False,True,2.19,0
4,True,1,2.0,0.0,16.04,30.0,0.0,170000.0,True,False,...,False,False,False,True,20,0,False,True,2.19,0


In [74]:
X_test.head()

,literate,attended_school,highest_school_lvl,preschool,finish_school_age,school_transportation_time,school_transportation_cost,school_expenses,is_student,not_attend_reason1_2.0,...,fam_role_7,fam_role_9,fam_role_14,sex,age,time_away,lives_with_partner,lives_with_parent,parents_education,num_parents_alive
0,True,1,5.0,3.0,16.11,15.0,0.0,130000.0,True,False,...,False,False,False,True,19,0,False,True,2.2,0
1,True,1,5.0,0.0,16.11,20.0,20000.0,230000.0,True,False,...,False,False,False,False,19,0,False,True,2.2,0
2,True,1,5.0,3.0,16.11,15.0,0.0,280000.0,True,False,...,False,False,False,False,21,0,False,True,2.2,0
3,True,1,5.0,0.0,16.11,15.0,0.0,113000.0,True,False,...,False,False,False,False,23,0,True,False,3.0,2
4,True,1,5.0,0.0,16.11,120.0,100000.0,100000.0,True,False,...,False,False,False,True,23,0,True,False,3.0,1


In [76]:
# Initialize the Random Forest model
rf = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=5, random_state=42)

# Train the Random Forest model on the training data
rf.fit(X, y)

# Make predictions on the test data
y_pred = rf.predict(X_test)

# Evaluate the model using accuracy
# accuracy = accuracy_score(y_test, y_pred)

# print(f"Accuracy on the test set: {accuracy:.2f}")


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- fam_role_14
- fam_role_5
Feature names seen at fit time, yet now missing:
- fam_role_10
- fam_role_6
- fam_role_8
- not_attend_reason1_11.0
- not_attend_reason1_5.0
- ...
